In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import pickle



In [2]:
# %%
# Load and process the data
df = pd.read_csv('Updated_Common_Data_Sains.csv')

# Convert date from 'YYYYMMDD' to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Extract features from date
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.dayofweek

# Drop the original date column
df = df.drop(columns=['date','own_brand'])



In [3]:

# %%
df.head()


,supermarket,Sains_price,unit,names,category,ASDA_price,Morrisons_price,Tesco_price,year,month,day,weekday
0,Sains,6.000,unit,Vuse Go,home,5.00,5.990000,2250.0,2024,1,29,0
1,Sains,6.000,unit,Vuse Go Edition,home,3.99,5.990000,2250.0,2024,1,29,0
2,Sains,5.425,kg,Heinz Tomato Ketchup,food_cupboard,4.80,5.666667,3.9,2024,1,29,0
3,Sains,7.000,kg,Bonne Maman Wild Blueberry Conserve,food_cupboard,9.70,7.700000,9.7,2024,1,29,0
4,Sains,8.900,kg,Dr. Oetker Liquid Glucose,food_cupboard,15.40,15.400000,15.4,2024,1,29,0


In [3]:
df.duplicated().sum()

0

In [5]:

# Encode categorical features
label_encoders = {}
for column in ['supermarket', 'names','unit','category']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Function to create sequences for LSTM
def create_sequences(group, sequence_length):
    X, y = [], []
    for i in range(len(group) - sequence_length):
        X.append(group.iloc[i:i + sequence_length].drop(columns='unit_price').values)
        y.append(group.iloc[i + sequence_length]['unit_price'])
    return np.array(X), np.array(y)

sequence_length = 10
X_list, y_list = [], []

# Group by the product or other unique identifier column
for _, group in df.groupby('names'):
    X_group, y_group = create_sequences(group, sequence_length)
    X_list.append(X_group)
    y_list.append(y_group)

# Concatenate all groups
X = np.concatenate(X_list)
y = np.concatenate(y_list)

# Ensure correct data types
X = X.astype(np.float32)
y = y.astype(np.float32)

In [6]:


# Split the data into training and testing sets
train_X,test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure correct data types for train and test sets
train_X = train_X.astype(np.float32)
train_y = train_y.astype(np.float32)
X_test =test_X.astype(np.float32)
test_y = test_y.astype(np.float32)

# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Build the enhanced LSTM model with additional layers
model = Sequential()

# First LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))

# Second LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Third LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Fourth LSTM layer with Dropout
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))

# First Dense layer
model.add(Dense(25, activation='relu'))

# Second Dense layer
model.add(Dense(10, activation='relu'))

# Output layer
model.add(Dense(1))

# Compile the model with the custom RMSE loss function
model.compile(optimizer='adam', loss=rmse)

# Train the model
model.fit(train_X, train_y, epochs=100, batch_size=64, validation_split=0.1)

# Evaluate the model
loss = model.evaluate(X_test, test_y)
print(f"Test RMSE: {loss}")



Epoch 1/100
158/158 [==============================] - 18s 45ms/step - loss: 28.5501 - val_loss: 25.5783
Epoch 2/100
158/158 [==============================] - 6s 37ms/step - loss: 26.5422 - val_loss: 27.0769
Epoch 3/100
158/158 [==============================] - 6s 37ms/step - loss: 26.1517 - val_loss: 26.0977
Epoch 4/100
158/158 [==============================] - 6s 39ms/step - loss: 25.8747 - val_loss: 25.9389
Epoch 5/100
158/158 [==============================] - 6s 38ms/step - loss: 26.1900 - val_loss: 26.1957
Epoch 6/100
158/158 [==============================] - 7s 42ms/step - loss: 25.6182 - val_loss: 26.8730
Epoch 7/100
158/158 [==============================] - 8s 50ms/step - loss: 25.9824 - val_loss: 25.6866
Epoch 8/100
158/158 [==============================] - 7s 43ms/step - loss: 25.7385 - val_loss: 26.0378
Epoch 9/100
158/158 [==============================] - 6s 41ms/step - loss: 25.9379 - val_loss: 26.0117
Epoch 10/100
158/158 [==============================] - 6s 40ms

In [7]:

# Save the model
model.save('Sains_lstm_model1.h5')


# Load the model
model = load_model('Sains_lstm_model1.h5',compile=False)


import pickle

# Save label encoders
with open('Sains_label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)


# Load label encoders
with open('Sains_label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)



model.predict(X_test)


array([[10.604487 ],
       [ 9.506513 ],
       [10.5390415],
       ...,
       [10.017068 ],
       [ 9.689376 ],
       [10.279777 ]], dtype=float32)